In [1]:
import os
import csv
import sys
import tensorflow as tf
import keras
import keras.preprocessing.image
import json
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# keras_retinanet imports
from keras_retinanet import losses
from keras_retinanet import layers
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# sys.path.append('/root/amol/product_detection/keras-retinanet/keras_retinanet/preprocessing/')
# from csv_generator import CSVGenerator
# from ..models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [4]:
# # create annotations.csv
# directory = '/root/data/blender_data/augmented_data0/training/validation_low_rez//'
# labels = json.load(open(os.path.join(directory, 'labels.json')))
# annotations = []
# for label in labels:
#     if not isinstance(label, list):
#         for bbox in label['bboxes']:
#             annotations.append([label['path'], bbox[0], bbox[1], bbox[2], bbox[3], 'fish'])

# with open(os.path.join(directory, 'annotations.csv'), 'w') as csvfile:
#     writer = csv.writer(csvfile, delimiter=',')
#     for row in annotations:
#         writer.writerow(row)
    

In [5]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [6]:
def create_generators(annotations,classes,batch_size=1,val_annotations=''):
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(flip_x_chance=0.5)

    
    
    train_generator = CSVGenerator(
        annotations,
        classes,
        transform_generator=transform_generator,
        batch_size=batch_size
    )


    if val_annotations:
        validation_generator = CSVGenerator(
            val_annotations,
            classes,
            batch_size=batch_size
        )
    else:
        validation_generator = None
    

    return train_generator, validation_generator

In [7]:
def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model

In [8]:
def create_models(backbone_retinanet, backbone, num_classes, weights, multi_gpu=0, freeze_backbone=False):
    modifier = freeze_model if freeze_backbone else None

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, backbone=backbone, nms=False, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)

        # append NMS for prediction only
        classification   = model.outputs[1]
        detections       = model.outputs[2]
        boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
        detections       = layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
        prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    else:
        model            = model_with_weights(backbone_retinanet(num_classes, backbone=backbone, nms=True, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model   = model
        prediction_model = model

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

    return model, training_model, prediction_model

In [9]:
annotations = '/root/data/blender_data/augmented_data0/training/train_low_rez/annotations.csv'
# annotations = '/root/thomas/mask/data/annotations.csv'
classes = '/root/thomas/mask/data/classID.csv'
batch_size = 8
val_annotations = '/root/data/blender_data/augmented_data0/training/validation_low_rez/annotations.csv'
val_annotations = annotations
weights = download_imagenet('resnet50')


In [10]:
train_generator, validation_generator = create_generators(annotations, classes, batch_size, val_annotations)

In [11]:
model, training_model, prediction_model = create_models(backbone_retinanet=retinanet,
                                                        backbone='resnet50',
                                                        num_classes=train_generator.num_classes(),
                                                        weights=weights)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.


In [12]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 1e-5
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_scheduler = LearningRateScheduler(step_decay)

In [13]:
# save model
filepath = os.path.join('/root/data/models/retinanet/', 'model_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath)

In [14]:
# start training
history = training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=1000//batch_size,
        epochs=30,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=100//batch_size,
        callbacks=[checkpoint, lr_scheduler]
    )

Epoch 1/30
125/125 [==============================] - 122s 976ms/step - loss: 4.6078 - regression_loss: 3.6457 - classification_loss: 0.9620 - val_loss: 3.1264 - val_regression_loss: 2.7296 - val_classification_loss: 0.3967
Epoch 2/30
125/125 [==============================] - 117s 939ms/step - loss: 2.5041 - regression_loss: 2.1994 - classification_loss: 0.3048 - val_loss: 1.9100 - val_regression_loss: 1.6538 - val_classification_loss: 0.2563
Epoch 3/30
125/125 [==============================] - 117s 939ms/step - loss: 1.6674 - regression_loss: 1.4311 - classification_loss: 0.2363 - val_loss: 1.4941 - val_regression_loss: 1.2773 - val_classification_loss: 0.2168
Epoch 4/30
125/125 [==============================] - 118s 943ms/step - loss: 1.3559 - regression_loss: 1.1571 - classification_loss: 0.1988 - val_loss: 1.2612 - val_regression_loss: 1.0816 - val_classification_loss: 0.1796
Epoch 5/30
125/125 [==============================] - 118s 941ms/step - loss: 1.1702 - regression_loss: 